## Terraforming Mars Cost-Benefit Analysis
<br>
Terraforming Mars is a competitive board game developed by Jacob Fryxelius and published by FryxGames in 2016. Players take on the role of corporations tasked with making Mars habitable through the raising of the oxygen level, temperature, and ocean coverage, as well as sundry other projects to progress humanity.

Currently sitting at #4 on BoardGameGeek's ranked board game list, Terraforming Mars has found broad popularity among strategy board gamers in large part due to its variety and complexity. With 208 unique cards and 12 distinct corporations, players are put into an endless assortment of scenarios. This incredible depth means that players must often intuit their next move rather than deduce it from values and probabilities.

In a bid to add more deduction back into the equation, this analysis aims to take every card in Terraforming Mars and boil it down to a single value in credits (the game's main resource) for simple value-comparison.

Let's get started.

In [20]:
import numpy as np
from statistics import median
import pandas as pd
import seaborn as sns
import csv
import ast
import math
from ipynb.fs.defs.TM_lib import vars_scan, pers, ors

np.set_printoptions(threshold=np.inf)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

The base_cards database, populated from a hand-entered csv, follows a couple of conventions:
* When a resource is written in uppercase it refers to that resource's production, whereas when written in lowercase it refers to the resource iself.
* All nouns are written in the singular in order to simplify pandas referencing.

The all_games database is a collection of stats from over 12,000 individually logged games downloaded from Simeon Simeonov's excellent [Terraforming Mars website](https://ssimeonoff.github.io/). The base_games database contains only the matches from all_games which were played with the full base version of the game.

In [2]:
# create databases
base_cards = pd.read_csv('TM_base_project_list.csv')
all_games = pd.read_csv('games_list.csv')
base_games = all_games[(all_games.expansions == "['CORPORATE']") & (all_games.map == "THARSIS")]

First, an average number of generations for each player count must be derived from base_games. These will be used to determine the income which will be generated over the remainder of the game for a given card after the generation during which it is played. They'll also be used to split the game into four quarters to illustrate how a card's value changes over time.

In [3]:
# determine generation averages per player count
last_gen_2p = int(round(base_games[base_games.players == 2].generations.mean()))
last_gen_3p = int(round(base_games[base_games.players == 3].generations.mean()))
last_gen_4p = int(round(base_games[base_games.players == 4].generations.mean()))
last_gen_5p = int(round(base_games[base_games.players == 5].generations.mean()))

last_gens = {
            'last_gen_2p': int(round(base_games[base_games.players == 2].generations.mean())),
            'last_gen_3p': int(round(base_games[base_games.players == 3].generations.mean())),
            'last_gen_4p': int(round(base_games[base_games.players == 4].generations.mean())),
            'last_gen_5p': int(round(base_games[base_games.players == 5].generations.mean()))
           }

print(f'''
Average length of game in generations:
2p: {last_gen_2p}
3p: {last_gen_3p}
4p: {last_gen_4p}
5p: {last_gen_5p}
''')

# average game length split into quarters with the median generation taken from each quarter
# X players, quarter N, median generation
gen_quarters = {
               '2p_q1_gen':round(last_gen_2p/4/2),
               '2p_q2_gen':round(last_gen_2p/4/2 + last_gen_2p/4),
               '2p_q3_gen':round(last_gen_2p/4/2 + last_gen_2p/4*2),
               '2p_q4_gen':round(last_gen_2p/4/2 + last_gen_2p/4*3),
               '3p_q1_gen':round(last_gen_3p/4/2),
               '3p_q2_gen':round(last_gen_3p/4/2 + last_gen_3p/4),
               '3p_q3_gen':round(last_gen_3p/4/2 + last_gen_3p/4*2),
               '3p_q4_gen':round(last_gen_3p/4/2 + last_gen_3p/4*3),
               '4p_q1_gen':round(last_gen_4p/4/2),
               '4p_q2_gen':round(last_gen_4p/4/2 + last_gen_4p/4),
               '4p_q3_gen':round(last_gen_4p/4/2 + last_gen_4p/4*2),
               '4p_q4_gen':round(last_gen_4p/4/2 + last_gen_4p/4*3),
               '5p_q1_gen':round(last_gen_5p/4/2),
               '5p_q2_gen':round(last_gen_5p/4/2 + last_gen_5p/4),
               '5p_q3_gen':round(last_gen_5p/4/2 + last_gen_5p/4*2),
               '5p_q4_gen':round(last_gen_5p/4/2 + last_gen_5p/4*3)
              }
keys_2p_gen = ['2p_q1_gen', '2p_q2_gen', '2p_q3_gen', '2p_q4_gen']
keys_3p_gen = ['3p_q1_gen', '3p_q2_gen', '3p_q3_gen', '3p_q4_gen']
keys_4p_gen = ['4p_q1_gen', '4p_q2_gen', '4p_q3_gen', '4p_q4_gen']
keys_5p_gen = ['5p_q1_gen', '5p_q2_gen', '5p_q3_gen', '5p_q4_gen']

print(f'''
Generation quarters:
2p: {str([gen_quarters.get(key) for key in keys_2p_gen]).strip('[]')}
3p: {str([gen_quarters.get(key) for key in keys_3p_gen]).strip('[]')}
4p: {str([gen_quarters.get(key) for key in keys_4p_gen]).strip('[]')}
5p: {str([gen_quarters.get(key) for key in keys_5p_gen]).strip('[]')}
''')



Average length of game in generations:
2p: 13
3p: 11
4p: 10
5p: 10


Generation quarters:
2p: 2, 5, 8, 11
3p: 1, 4, 7, 10
4p: 1, 4, 6, 9
5p: 1, 4, 6, 9



Next, each resource in the game must be converted to a number of credits. The [standard projects](https://3.bp.blogspot.com/-fs7vQZGRFs4/XAjAYOSBspI/AAAAAAAAFSA/kInbyuaLoLc0PANwJeFDpJditT5Ims3_wCLcBGAs/s1600/terraforming-mars-standard-projects-1.jpg) are set as listed on the game board, and steel and titanium as listed on the [player boards](https://cf.geekdo-images.com/medium/img/qa7YwBE6pc-ZR0relFRrzUAlQXw=/fit-in/500x500/filters:no_upscale()/pic2891980.jpg).  

Plants and heat are set by dividing 8, the number of plants/heat needed to equal a temp/greenery (also illustrated on the player boards), into the credit values for the temp and greenery standard projects.  

For energy, it was necessary to borrow some information from one of the expansions: _Colonies_. In order to trade with a colony, players must spend either 9 credits, 3 titanium, or 3 energy. This suggests that 1 energy = 1 titanium = 3 credits.  

Drawing a card is valued at 4 credits, combining the price to buy a card (3 credits) with the 1 credit gained from discarding it using the "Sell patents" standard project.

In [4]:
# variable definitions

income = 0
vp = 0
TR = vp + income

# standard projects
temp = 14
ocean = 18
greenery = 23
def city(income):
    return 25 - (income + 1)
oxygen = TR

# resources
credit = 1
steel = 2
titanium = 3
plant = greenery / 8
heat = temp / 8
energy = 3
draw = 4

# production
def CREDIT(income):
    return income + 1
def STEEL(income):
    return 2 * income
def TITANIUM(income):
    return 3 * income
def PLANT(income):
    return greenery/8 * (income + 1)
def ENERGY(income):
    return 3 * income
def HEAT(income):
    return (temp/8) * income

vars_dict_static = {'temp':temp, 'ocean':ocean, 'greenery':greenery, 'credit':credit, 'steel':steel, 
                    'titanium':titanium, 'plant':plant, 'heat':heat, 'energy':energy, 'draw':draw}

vars_dict_variable = {'city':city, 'CREDIT':CREDIT, 'STEEL':STEEL, 'TITANIUM':TITANIUM, 
                      'PLANT':PLANT, 'ENERGY':ENERGY, 'HEAT':HEAT}

vars_dict_resource = {}

In [5]:
# determine number of remaining generations
def remgen(generation, player_count):
    global income
    for players in last_gens:
        if str(player_count) in players:
            last_gen = last_gens[players]
    income = last_gen - generation
    return income, last_gen

#### Tile placement

Placing a tile on an area with resources allows the player to take those resources, adding to the value of the tile. To determine the average value added from these map resources, the total sum of credits gained from all of the resources on the map are divided by the total number of areas. This is done separately for ocean areas and non-ocean areas. The two off-world areas and the Noctis City area have been disregarded, since they cannot be placed upon except for unique circumstances.

<img src="https://4.bp.blogspot.com/-KVleOqhdyg0/XBxty1rpALI/AAAAAAAAFac/iJsDNEW0SDw6gTK2aHIS0oLc9px5XvzjQCEwYBhgL/s1600/TM%2BOriginal-Mars%2BOnly.jpg" alt="TM map" width="500"/>

In [6]:
#determine added value from gained resources when placing a tile

tile_land = (
            (2*steel * 3 +
            draw * 3 +
            steel * 3 +
            plant + titanium +
            plant * 10 +
            2*plant * 7 +
            titanium)
            / 48)

tile_ocean = (
             (2*steel +
             draw +
             plant * 3 +
             2*plant * 4 +
             2*titanium)
             / 12)

print(f'Placing a land tile is worth {str(round(tile_land, 3))} credits.')
print(f'Placing an ocean tile is worth {str(round(tile_ocean, 3))} credits.')

Placing a land tile is worth 2.247 credits.
Placing an ocean tile is worth 3.802 credits.


Victory points (VP) must also be converted to credits. To do this, the three terraforming standard projects were used. Since they confer a terraforming rating (TR) and not a victory point, the extra income from the TR must be subtracted from the cost of the project. This means that similar to the cards themselves, the value of a VP changes over time.

In addition to the generated income, the average bonus credits accrued through tile placement are also factored into the Aquifer and Greenery standard projects when calculating for VP value.

To come to a single value for a given generation and player count, the "points per credit" are determined for each of the three standard terraforming projects and then the average is taken. Note that greeneries are worth two points (one for the TR from raising the oxygen, another from the greenery tile itself) and so are calculated accordingly.

In [7]:
# define credits per victory point
def credits_per_vp(generation, player_count):
    income, last_gen = remgen(generation, player_count)
    temp_ppc = 1/(temp - income) # ppc = points per credit
    ocean_ppc = 1/(ocean - income - tile_ocean)
    greenery_ppc = 2/(greenery - income - tile_land)
    ppc = (temp_ppc + ocean_ppc + greenery_ppc)/4
    vp = 1/ppc
    return vp

# numbers of credits per victory point for each player count and median generation
# X players, quarter N, credits per point
gen_credits_per_vp = {'2pq1cpp':credits_per_vp(2, 2),
                      '2pq2cpp':credits_per_vp(5, 2),
                      '2pq3cpp':credits_per_vp(8, 2),
                      '2pq4cpp':credits_per_vp(11, 2),
                      '3pq1cpp':credits_per_vp(1, 3),
                      '3pq2cpp':credits_per_vp(4, 3),
                      '3pq3cpp':credits_per_vp(7, 3),
                      '3pq4cpp':credits_per_vp(10, 3),
                      '4pq1cpp':credits_per_vp(1, 4),
                      '4pq2cpp':credits_per_vp(4, 4),
                      '4pq3cpp':credits_per_vp(6, 4),
                      '4pq4cpp':credits_per_vp(9, 4),
                      '5pq1cpp':credits_per_vp(1, 5),
                      '5pq2cpp':credits_per_vp(4, 5),
                      '5pq3cpp':credits_per_vp(6, 5),
                      '5pq4cpp':credits_per_vp(9, 5)
                     }

keys2pcpp = ['2pq1cpp', '2pq2cpp', '2pq3cpp', '2pq4cpp']
keys3pcpp = ['3pq1cpp', '3pq2cpp', '3pq3cpp', '3pq4cpp']
keys4pcpp = ['4pq1cpp', '4pq2cpp', '4pq3cpp', '4pq4cpp']
keys5pcpp = ['5pq1cpp', '5pq2cpp', '5pq3cpp', '5pq4cpp']

print(f'''
Generation credits per point per quarter:
   q1, q2, q3, q4
2p: {str([round(gen_credits_per_vp.get(key)) for key in keys2pcpp]).strip('[]')}
3p: {str([round(gen_credits_per_vp.get(key)) for key in keys3pcpp]).strip('[]')}
4p: {str([round(gen_credits_per_vp.get(key)) for key in keys4pcpp]).strip('[]')}
5p: {str([round(gen_credits_per_vp.get(key)) for key in keys5pcpp]).strip('[]')}
''')


Generation credits per point per quarter:
   q1, q2, q3, q4
2p: 5, 8, 12, 15
3p: 6, 9, 13, 16
4p: 7, 10, 13, 16
5p: 7, 10, 13, 16



In [8]:
#run above functions and determine credits per TR
def initialize(generation, player_count):
    income, last_gen = remgen(generation, player_count)
    vp = credits_per_vp(generation, player_count)
    TR = vp + income
    return income, vp, TR, last_gen

In [9]:
# chance to draw each tag

# Earth
earth_tags = base_cards[base_cards.Tags.str.contains('earth', na=False)]
chance_earth_played = len(earth_tags) / len(base_cards)
chance_earth_showing = len(earth_tags[~earth_tags.Tags.str.contains('event', na=False)]) \
                       / len(base_cards)
# Science
science_tags = base_cards[base_cards.Tags.str.contains('science', na=False)]
chance_science_played = len(science_tags) / len(base_cards)
chance_science_showing = len(science_tags[~science_tags.Tags.str.contains('event', na=False)]) \
                       / len(base_cards)
# Plant
plant_tags = base_cards[base_cards.Tags.str.contains('plant', na=False)]
chance_plant_played = len(plant_tags) / len(base_cards)
chance_plant_showing = len(plant_tags[~plant_tags.Tags.str.contains('event', na=False)]) \
                       / len(base_cards)
# Microbe
microbe_tags = base_cards[base_cards.Tags.str.contains('microbe', na=False)]
chance_microbe_played = len(microbe_tags) / len(base_cards)
chance_microbe_showing = len(microbe_tags[~microbe_tags.Tags.str.contains('event', na=False)]) \
                       / len(base_cards)
# Animal
animal_tags = base_cards[base_cards.Tags.str.contains('animal', na=False)]
chance_animal_played = len(animal_tags) / len(base_cards)
chance_animal_showing = len(animal_tags[~animal_tags.Tags.str.contains('event', na=False)]) \
                       / len(base_cards)
# Space
space_tags = base_cards[base_cards.Tags.str.contains('space', na=False)]
chance_space_played = len(space_tags) / len(base_cards)
chance_space_showing = len(space_tags[~space_tags.Tags.str.contains('event', na=False)]) \
                       / len(base_cards)
# Event
event_tags = base_cards[base_cards.Tags.str.contains('event', na=False)]
chance_event_played = len(event_tags) / len(base_cards)

# Building
building_tags = base_cards[base_cards.Tags.str.contains('building', na=False)]
chance_building_played = len(building_tags) / len(base_cards)
chance_building_showing = len(building_tags[~building_tags.Tags.str.contains('event', na=False)]) \
                       / len(base_cards)
# Jovian
jovian_tags = base_cards[base_cards.Tags.str.contains('jovian', na=False)]
chance_jovian_played = len(jovian_tags) / len(base_cards)
chance_jovian_showing = len(jovian_tags[~jovian_tags.Tags.str.contains('event', na=False)]) \
                       / len(base_cards)
# Power
power_tags = base_cards[base_cards.Tags.str.contains('power', na=False)]
chance_power_played = len(power_tags) / len(base_cards)
chance_power_showing = len(power_tags[~power_tags.Tags.str.contains('event', na=False)]) \
                       / len(base_cards)
# City
city_tags = base_cards[base_cards.Tags.str.contains('city', na=False)]
chance_city_played = len(city_tags) / len(base_cards)
chance_city_showing = len(city_tags[~city_tags.Tags.str.contains('event', na=False)]) \
                       / len(base_cards)

tags_played_dict = {'earth':chance_earth_played, 'science':chance_science_played, 
                    'plant':chance_plant_played, 'microbe':chance_microbe_played, 
                    'animal':chance_animal_played, 'space':chance_space_played,
                    'event':chance_event_played, 'building':chance_building_played, 
                    'jovian':chance_jovian_played,'power':chance_power_played, 
                    'city':chance_city_played}

tags_showing_dict = {'earth':chance_earth_showing, 'science':chance_science_showing, 
                    'plant':chance_plant_showing, 'microbe':chance_microbe_showing, 
                    'animal':chance_animal_showing, 'space':chance_space_showing,
                    'event':chance_event_played, 'building':chance_building_showing, 
                    'jovian':chance_jovian_showing,'power':chance_power_showing, 
                     'city':chance_city_showing}

def cards_current(generation):
    return 10 + (4 * (generation - 1))

#### Microbes  
The following cards are taken into account when calculating the value of a microbe:
* Ants - 1 vp per 2 microbes  
* Decomposers - 1 vp per 3 microbes  
* GHG Producing Bacteria - 1 temp per 2 microbes  
* Nitrite Reducing Bacteria - 1 TR per 3 microbes  
* Regolith Eaters - 1 oxygen per 2 microbes  
* Tardigrades - 1 vp per 4 microbes  

In [10]:
# microbe value
# microbe_tags = base_cards[base_cards.Tags.str.contains('microbe', na=False)]
# display(microbe_tags)

def microbe_val(generation, player_count):
    income, vp, TR, last_gen = initialize(generation, player_count)
    
    ants = vp / 2
    decomposers = vp / 3
    ghg_producing_bacteria = temp / 2
    nitrite_reducing_bacteria = TR / 3
    regolith_eaters = greenery / 4
    tardigrades = vp / 4
    
    microbes_list = [ants, decomposers, ghg_producing_bacteria, nitrite_reducing_bacteria,
                     regolith_eaters, tardigrades]
    microbe = sum(microbes_list) / len(microbes_list)
    return microbe

vars_dict_resource.update({'microbe':microbe_val})

print(f'A microbe is worth {str(round(microbe_val(1, 2), 3))} credits in generation 1 with 2 players.')

A microbe is worth 3.59 credits in generation 1 with 2 players.


#### Animals

The following cards are taken into account when calculating the value of an animal:  
* Birds - 1 vp per animal
* Fish - 1 vp per animal
* Livestock - 1 vp per animal
* Predators - 1 vp per animal
* Ecological Zone - 1 vp per 2 animals
* Herbivores - 1 vp per 2 animals
* Pets - 1 vp per 2 animals
* Small Animals - 1 vp per 2 animals

In [11]:
# animal value
# animal_tags = base_cards[base_cards.Tags.str.contains('animal', na=False)]
# display(animal_tags)

def animal_val(generation, player_count):
    income, vp, TR, last_gen = initialize(generation, player_count)
    
    birds= fish= livestock= predators = vp
    ecological_zone= herbivores= pets= small_animals = vp/2
    
    animals_list = [birds, fish, livestock, predators, ecological_zone,
                    herbivores, pets, small_animals]
    animal = sum(animals_list) / len(animals_list)
    return animal

vars_dict_resource.update({'animal':animal_val})

print(f'An animal is worth {str(round(animal_val(1, 2), 3))} credits in generation 1 with 2 players.')

An animal is worth 2.535 credits in generation 1 with 2 players.


This brings me to the point of actually calculating the card values. My plan is to write a main function for doing this, then running it for most of the cards to create a values database.

Beyond that I've set aside fifteen active cards that have a cost to use. Ironworks, for example, costs 4 energy to use and grants a steel and raises the oxygen by 1. Do I calculate cards such as these as if they would be used in every remaining generation? That's probably what I'll do, I just haven't decided yet.

Finally there is a separate group of 23 cards that I don't know how to handle. An example would be Advanced Alloys, which increases the value of steel and titanium by one credit each. I'll probably have to have a separate function for each of those. Hopefully I'll be able to even figure out how to put numbers to each of them.

Before I start writing the probably very long function to determine the value of most of the cards, I will try a few card-specific functions just to get some practice and ideas about how to write the big one.

In [12]:
# Food Factory test
def food_factory(generation, player_count):
    income, vp, TR, last_gen = initialize(generation, player_count)
    
    value = 0
    value -= 12 #card cost
    value -= round(plant*(income+1))
    value += round(4*income)
    value += vp
    display(f'The value of Food Factory in gen {str(generation)} with {str(player_count)} players is {str(round(value))} credits')
food_factory(1, 2)

'The value of Food Factory in gen 1 with 2 players is 2 credits'

In [13]:
# define special case cards to be left out of the main function
special_cases = ['Adaptation Technology', 'Advanced Alloys', 'Anti-Gravity Technology', 'Ants', 
                 'Arctic Algae', 'Business Network', 'Capital', 'CEO’s Favorite Project', 
                 'Commercial District', 'Decomposers', 'Earth Catapult', 'Earth Office', 
                 'Ecological Zone', 
                 'Ganymede Colony', 'GHG Producing Bacteria', 'Herbivores', 'Immigrant City', 
                 'Immigration Shuttles', 
                 'Indentured Workers', 'Insulation', 'Inventor’s Guild', 'Io Mining Industries', 
                 'Land Claim', 
                 'Lava Flows', 'Mars University', 'Martian Rails', 'Mass Converter', 'Media Group', 
                 'Mining Area', 'Mining Rights', 'Nitrite Reducing Bacteria', 
                 'Nitrogen-Rich Asteroid', 'Noctis City', 'Olympus Conference', 'Optimal Aerobraking', 
                 'Pets', 'Phobos Space Haven', 'Power Infrastructure',
                 'Predators', 'Protected Habitat', 'Quantum Extractor', 'Regolith Eaters', 
                 'Research Outpost', 
                 'Robotic Workforce', 'Rover Construction', 'Search for Life', 'Shuttles', 
                 'Space Station', 'Special Design', 
                 'Standard Technology', 'Viral Enhancers', 'Water Import from Europa']

#print(len(special_cases))
base_cases = base_cards[~base_cards.Title.isin(special_cases)].copy().reset_index(drop=True)
#print(len(base_cases) + len(special_cases))

In [14]:
# main value calculation function
def is_null(x):
    return x is None or (isinstance(x, float) and math.isnan(x))


def compute_additional_cost_value(add_cost, generation, player_count):
    if is_null(add_cost):
        return 0
    value = 0
    for idx, ele in enumerate(add_cost.split(',')):
        add_cost_list = ele.strip().split(' ')
        value -= vars_scan(add_cost_list, generation, player_count)
    return value


def compute_victory_points_value(victory, vp):
    if is_null(victory) or not victory.isdigit():
        return 0
    return int(victory) * vp


def compute_immediate_benefit_value(i_benefit, generation, player_count):
    if is_null(i_benefit):
        return 0

    value = 0

    for idx, ele in enumerate(i_benefit.split(',')):
        i_benefit_list = ele.strip().split(' ')

        # for benefits dependent on the amount of another variable
        if 'per' in i_benefit_list:
            value += pers(i_benefit_list, generation, player_count)

        # for benefits with a choice; the larger of the two values is taken
        elif 'or' in i_benefit_list:
            value += ors((' ').join(i_benefit_list), generation, player_count)

        # for ocean and greenery tiles played on the opposite site
        elif 'land' in i_benefit_list and 'ocean' in i_benefit_list:
            value += ocean + tile_land - tile_ocean
        elif 'ocean' in i_benefit_list and 'greenery' in i_benefit_list:
            value += greenery + tile_ocean - tile_land

        # for special tiles
        elif 'special' in i_benefit_list:
            if 'ocean' in i_benefit_list:
                value += tile_ocean
            else:
                value += tile_land

         # for simple added values
        else:
            value += vars_scan(i_benefit_list, generation, player_count)

    return value


def compute_active_cost_benefit_value(a_cost, a_benefit, victory, generation, last_gen, player_count):
    if is_null(a_cost) or is_null(a_benefit):
        # Note: If one is null then the other must also be null
        return 0

    value = 0
    current_gen = generation

    while current_gen <= last_gen:
        # Cost
        cost = 0
        act_cost_list = a_cost.split(' ')
        if 'or' in a_cost:
            cost = ors(a_cost, current_gen, player_count, cost=True)
        else:
            is_variable_benefit = any(
                x for x in a_benefit.split(' ') if x in vars_dict_variable
            )
            if not is_variable_benefit or (is_variable_benefit and current_gen < last_gen):
                cost = vars_scan(act_cost_list, current_gen, player_count)

        # Benefit
        benefit = 0
        for benefit_list in a_benefit.split(','):
            benefit_list_split = benefit_list.strip().split(' ')
            if 'or' in benefit_list:
                benefit += ors(benefit_list, current_gen, player_count)
            elif 'on card' in benefit_list:
                number = (int(victory[0]) / int(victory[6])) * int(benefit_list[0])
                current_vp = credits_per_vp(current_gen, player_count)
                benefit += number * current_vp
            else:
                variable_eles = set(vars_dict_variable) & {'CREDIT', 'PLANT'}
                is_variable_benefit = any(x for x in benefit_list_split if x in variable_eles)
                if not is_variable_benefit or (is_variable_benefit and current_gen < last_gen):
                    # ensures increased production on the final turn isn't converted into points
                    benefit += vars_scan(benefit_list_split, current_gen, player_count)

        if cost < benefit:
            value += benefit - cost

        current_gen += 1

    return value


def compute_removal_value(removed, generation, player_count):
    if is_null(removed):
        return 0
    
    value = 0
    
    if 'or' in removed:
        value += ors(removed, generation, player_count)
    else:
        value += vars_scan(removed.split(' '), generation, player_count)
    return value


def compute_passive_benefit_value(p_benefit, generation):
    if is_null(p_benefit):
        return 0
    value = 0
    p_benefit_list = p_benefit.split(' ')
    cards_to_play = sum(ave_cards_played[(generation-1):last_gen])
    if not 'tag' in p_benefit_list:
        value = (cards_to_play - 1) * int(p_benefit_list[2])
    elif 'tag' in p_benefit_list:
        for idx, ele in enumerate(p_benefit_list):
            if ele in tags_played_dict:
                value = 4 * (last_gen - generation + 1) * tags_played_dict[ele] * int(p_benefit_list[idx+3])
    return value


def base_value(func):
    def wrapper(database, generation, player_count):
        def compute_value(row):
            income, vp, TR, last_gen = initialize(generation, player_count)
            
            # Primary Cost
            value = -row.at['Primary_Cost']

            # Additional Cost
            add_cost = row.at['Additional_Cost']
            value += compute_additional_cost_value(add_cost, generation, player_count)

            # Victory Points
            victory = row.at['Victory_Points']
            value += compute_victory_points_value(victory, vp)

            # Immediate Benefit
            i_benefit = row.at['Immediate_Benefit']
            value += compute_immediate_benefit_value(i_benefit, generation, player_count)
            
            # Passive Benefit
            p_benefit = row.at['Passive_Benefit']

            # Active Cost/Benefit
            a_cost = row.at['Active_Cost']
            a_benefit = row.at['Active_Benefit']
            value += compute_active_cost_benefit_value(a_cost, a_benefit, victory, generation, last_gen, player_count)

            # Removed from Opponent
            removed = row.at['Removed_from_Opponent']
            value += compute_removal_value(removed, generation, player_count)

            # Special Cases
            value += func(database, generation, player_count, row, last_gen)
            
            return round(value)
    
        new_database = database
        new_database['Value'] = database.apply(compute_value, axis=1)
        new_database['Generation'] = generation
        new_database['Players'] = player_count
            
        return new_database
    return wrapper


In [17]:
### DEBUGGER
# main value calculation function
def is_null(x):
    return x is None or (isinstance(x, float) and math.isnan(x))

def compute_primary_cost_value(p_cost):
    value = 0
    value += p_cost
    print('Primary Cost = ', p_cost)
    return value

def compute_additional_cost_value(add_cost, generation, player_count):
    if is_null(add_cost):
        return 0
    value = 0
    for idx, ele in enumerate(add_cost.split(',')):
        add_cost_list = ele.strip().split(' ')
        value -= vars_scan(add_cost_list, generation, player_count)
    print('Additional Cost = ', value)
    return value


def compute_victory_points_value(victory, vp):
    if is_null(victory) or len(victory) > 3:
        return 0
    print('Victory Points = ', int(victory) * vp)
    return int(victory) * vp


def compute_immediate_benefit_value(i_benefit, generation, player_count):
    if is_null(i_benefit):
        return 0

    value = 0

    for idx, ele in enumerate(i_benefit.split(',')):
        i_benefit_list = ele.strip().split(' ')

        # for benefits dependent on the amount of another variable
        if 'per' in i_benefit_list:
            value += pers(i_benefit_list, generation, player_count)

        # for benefits with a choice; the larger of the two values is taken
        elif 'or' in i_benefit_list:
            value += ors((' ').join(i_benefit_list), generation, player_count)

        # for ocean and greenery tiles played on the opposite site
        elif 'land' in i_benefit_list and 'ocean' in i_benefit_list:
            value += ocean + tile_land - tile_ocean
        elif 'ocean' in i_benefit_list and 'greenery' in i_benefit_list:
            value += greenery + tile_ocean - tile_land

        # for special tiles
        elif 'special' in i_benefit_list:
            if 'ocean' in i_benefit_list:
                value += tile_ocean
            else:
                value += tile_land

         # for simple added values
        else:
            value += vars_scan(i_benefit_list, generation, player_count)
    print('Immediate Benefit = ', value)
    return value


def compute_active_cost_benefit_value(a_cost, a_benefit, victory, generation, last_gen, player_count):
    if is_null(a_cost) and is_null(a_benefit):
        return 0

    value = 0
    current_gen = generation

    while current_gen <= last_gen:
        # Cost
        cost = 0
        if is_null(a_cost):
            pass
        else:
            act_cost_list = a_cost.split(' ')
            if 'or' in a_cost:
                cost = ors(a_cost, current_gen, player_count, cost=True)
            else:
                is_variable_benefit = any(
                    x for x in a_benefit.split(' ') if x in vars_dict_variable
                )
                if not is_variable_benefit or (is_variable_benefit and current_gen < last_gen):
                    cost = vars_scan(act_cost_list, current_gen, player_count)

        # Benefit
        benefit = 0
        for benefit_list in a_benefit.split(','):
            benefit_list_split = benefit_list.strip().split(' ')
            if 'or' in benefit_list:
                benefit += ors(benefit_list, current_gen, player_count)
            elif 'on card' in benefit_list:
                number = (int(victory[0]) / int(victory[6])) * int(benefit_list[0])
                current_vp = credits_per_vp(current_gen, player_count)
                benefit += number * current_vp
            else:
                variable_eles = set(vars_dict_variable) & {'CREDIT', 'PLANT'}
                is_variable_benefit = any(x for x in benefit_list_split if x in variable_eles)
                if not is_variable_benefit or (is_variable_benefit and current_gen < last_gen):
                    # ensures increased production on the final turn isn't converted into points
                    benefit += vars_scan(benefit_list_split, current_gen, player_count)

        if cost < benefit:
            value += benefit - cost

        current_gen += 1
    print('Active Cost/Benefit = ', value)
    return value


def compute_removal_value(removed, generation, player_count):
    if is_null(removed):
        return 0
    
    value = 0
    
    if 'or' in removed:
        value += ors(removed, generation, player_count)
    else:
        value += vars_scan(removed.split(' '), generation, player_count)
    print('Removal from Opponents = ', value)
    return value


#def compute_passive_benefit_value(p_benefit, generation):
#    if is_null(p_benefit):
#        return 0
#    value = 0
#    p_benefit_list = p_benefit.split(' ')
#    cards_to_play = sum(ave_cards_played[(generation-1):last_gen])
#    if not 'tag' in p_benefit_list:
#        value = (cards_to_play - 1) * int(p_benefit_list[2])
#    elif 'tag' in p_benefit_list:
#        for idx, ele in enumerate(p_benefit_list):
#            if ele in tags_played_dict:
#                value = 4 * (last_gen - generation + 1) * tags_played_dict[ele] * int(p_benefit_list[idx+3])
#    return value


def base_value(func):
    def wrapper(database, generation, player_count):
        def compute_value(row):
            income, vp, TR, last_gen = initialize(generation, player_count)
            
            # Primary Cost
            p_cost = -row.at['Primary_Cost']
            value = compute_primary_cost_value(p_cost)

            # Additional Cost
            add_cost = row.at['Additional_Cost']
            value += compute_additional_cost_value(add_cost, generation, player_count)

            # Victory Points
            victory = row.at['Victory_Points']
            value += compute_victory_points_value(victory, vp)

            # Immediate Benefit
            i_benefit = row.at['Immediate_Benefit']
            value += compute_immediate_benefit_value(i_benefit, generation, player_count)
            
            # Passive Benefit
            p_benefit = row.at['Passive_Benefit']

            # Active Cost/Benefit
            a_cost = row.at['Active_Cost']
            print('active cost = ', row.at['Active_Cost'])
            a_benefit = row.at['Active_Benefit']
            value += compute_active_cost_benefit_value(a_cost, a_benefit, victory, generation, last_gen, player_count)

            # Removed from Opponent
            removed = row.at['Removed_from_Opponent']
            value += compute_removal_value(removed, generation, player_count)

            # Special Cases
            value += func(database, generation, player_count, row, last_gen)
            
            return round(value)
    
        new_database = database
        new_database['Value'] = database.apply(compute_value, axis=1)
        new_database['Generation'] = generation
        new_database['Players'] = player_count
            
        return new_database
    return wrapper


In [18]:
@base_value
def base_case_tester(database, generation, player_count, *args):
    return 0
    
base_case_tester(base_cases, 11, 2)

Primary Cost =  -10
Immediate Benefit =  9
active cost =  nan
Primary Cost =  -9
Immediate Benefit =  8.625
active cost =  nan
Primary Cost =  -11
Victory Points =  44.12309757837058
active cost =  nan
Primary Cost =  -26
Immediate Benefit =  30.542524618817595
active cost =  nan
Primary Cost =  -21
Additional Cost =  -6
Victory Points =  14.707699192790194
active cost =  nan
Active Cost/Benefit =  24
Primary Cost =  -10
Immediate Benefit =  20.125
active cost =  nan
Primary Cost =  -18
active cost =  8 credit (steel may be spent)
Active Cost/Benefit =  30
Primary Cost =  -6
Immediate Benefit =  8.625
active cost =  nan
Primary Cost =  -15
Victory Points =  14.707699192790194
Immediate Benefit =  16.4453125
active cost =  nan
Primary Cost =  -12
Immediate Benefit =  12
active cost =  nan
Primary Cost =  -14
Immediate Benefit =  20
active cost =  nan
Removal from Opponents =  8.625
Primary Cost =  -30
Victory Points =  29.41539838558039
Immediate Benefit =  12
active cost =  nan
Primary

,Title,Game,Type,Primary_Cost,Additional_Cost,Tags,Prerequisites,Victory_Points,Immediate_Benefit,Passive_Benefit,Active_Cost,Active_Benefit,Removed_from_Opponent,Value,Generation,Players
0,Acquired Company,corporate,automated,10,NaN,earth,NaN,NaN,3 CREDIT,NaN,NaN,NaN,NaN,-1,11,2
1,Adapted Lichen,base,automated,9,NaN,plant,NaN,NaN,1 PLANT,NaN,NaN,NaN,NaN,0,11,2
2,Advanced Ecosystems,base,automated,11,NaN,"plant, microbe, animal","1 plant, 1 microbe, and 1 animal tag",3,NaN,NaN,NaN,NaN,NaN,33,11,2
3,Aerobraked Ammonia Asteroid,base,event,26,NaN,"space, event",NaN,NaN,"2 microbe on other card, 3 HEAT, 1 PLANT",NaN,NaN,NaN,NaN,5,11,2
4,AI Central,corporate,active,21,1 ENERGY,"science, building",3 science tag,1,NaN,NaN,NaN,2 draw,NaN,12,11,2
5,Algae,base,automated,10,NaN,plant,5 ocean,NaN,"1 plant, 2 PLANT",NaN,NaN,NaN,NaN,10,11,2
6,Aquifer Pumping,base,active,18,NaN,building,NaN,NaN,NaN,NaN,8 credit (steel may be spent),1 ocean,NaN,12,11,2
7,Archaebacteria,base,automated,6,NaN,microbe,max -18°C,NaN,1 PLANT,NaN,NaN,NaN,NaN,3,11,2
8,Artificial Lake,base,automated,15,NaN,building,-6°C,1,1 ocean on land site,NaN,NaN,NaN,NaN,16,11,2
9,Artificial Photosynthesis,base,automated,12,NaN,science,NaN,NaN,1 PLANT or 2 ENERGY,NaN,NaN,NaN,NaN,0,11,2


In [19]:
# Single card tester
special_case_db = base_cards[base_cards.Title.isin(special_cases)].copy().reset_index(drop=True)

biomass_combustors = base_cases[base_cases.Title == 'Biomass Combustors'].copy()
display(biomass_combustors)

print('income, vp, TR, last gen = ', initialize(11, 2))
print('land tile = ', tile_land)
print('ocean tile = ', tile_ocean)
print('animal = ', animal_val(11, 2))
print('microbe = ', microbe_val(11, 2))
generation = 11
player_count = 2
vp_list = []
for players in last_gens:
    if str(player_count) in players:
        last_gen = last_gens[players]
for i in range(generation, last_gen + 1):
    vp_list.append(credits_per_vp(i, player_count))
print('rem gen vps = ', vp_list)

print('************')
base_case_tester(biomass_combustors, 11, 2)

,Title,Game,Type,Primary_Cost,Additional_Cost,Tags,Prerequisites,Victory_Points,Immediate_Benefit,Passive_Benefit,Active_Cost,Active_Benefit,Removed_from_Opponent,Value,Generation,Players
15,Biomass Combustors,base,automated,4,NaN,"power, building",6% oxygen,-1,2 ENERGY,NaN,NaN,NaN,1 PLANT,2,11,2


income, vp, TR, last gen =  (2, 14.707699192790194, 16.707699192790194, 13)
land tile =  2.2473958333333335
ocean tile =  3.8020833333333335
animal =  11.030774394592644
microbe =  5.7087623094087965
rem gen vps =  [14.707699192790194, 15.751437078285592, 16.790152134624012]
************
Primary Cost =  -4
Victory Points =  -14.707699192790194
Immediate Benefit =  12
active cost =  nan
Removal from Opponents =  8.625


,Title,Game,Type,Primary_Cost,Additional_Cost,Tags,Prerequisites,Victory_Points,Immediate_Benefit,Passive_Benefit,Active_Cost,Active_Benefit,Removed_from_Opponent,Value,Generation,Players
15,Biomass Combustors,base,automated,4,NaN,"power, building",6% oxygen,-1,2 ENERGY,NaN,NaN,NaN,1 PLANT,2,11,2


## General Notes -

* Ignoring opportunity cost. This is why a card draw is worth only 4. Certain cards will be slightly devalued due to this, such as Business Contacts (where the extra two cards drawn and then discarded are not tallied)  
* Greeneries and Cities. The map point from a greenery is used in calculating the value of victory points, but it is not added to the value of the greenery standard project. Likewise, the potential map value of a city tile is not added to the value of the city standard project. The reasoning for this is that those map points/potential map points have been factored into determining the cost of the standard project.
* Tile placement bonus. Average added values have been determined for placing a tile on a land area or on an ocean area based on the resource bonuses printed on some of those areas. These bonuses have been factored into the cost of the relevant standard projects.
* In most cases numbers are rounded only at the point of presenting the final card values.

#### Project Notes -
* In the `pers()` function in TM_lib.ipynb, the event portion is focused solely on Media Archives. Only "all events played" is handled. If I add any expansions later on, I will need to update this if any cards rely on the number of events played by the current player only.
* For concision, I've only distinguished between tags played and tags showing for those tags that have any event versions. If I add any expansions, I'll have to reassess.  
* First generation Birds is impossible but a value is still attached to it. This is probably not a big deal as I'm ignoring prerequisites. Getting first generation Ironworks will yield an oxygen every generation, even though there is a limited amount of oxygen and some of it will get raised by other players. This is a bigger problem I need to think about.